# Verodata preprocessing

In [1]:
import pandas as pd
import numpy as np
import requests
from json.decoder import JSONDecodeError

In [10]:
df = pd.read_csv('yhteiso_tuloverotus_julk_20{}.csv'.format(11), delimiter=';',encoding = "ISO-8859-1",
                decimal=',')
df.columns = df.columns.str.split('|', expand=True).droplevel(1).str.strip(' ')
#df = df[df['Verotettava tulo'] != 0]

for i in range(12, 19):
    df_t = pd.read_csv('yhteiso_tuloverotus_julk_20{}.csv'.format(i), delimiter=';',encoding = "ISO-8859-1",
                decimal=',')
    df_t.columns = df.columns
    #df_t = df_t[df_t['Verotettava tulo'] != 0]
    df = df.append(df_t)
    
#df = df[df.iloc[:,4:].abs().sum(1) != 0]
df.to_csv('verodata.csv')

In [3]:
df = pd.read_csv('verodata.csv', index_col=0)

C:\Users\lauri\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
records = pd.DataFrame()

In [4]:
i = 1500000
while i < 3000000:
    print('Trying: {}'.format(i))
    url = 'https://avoindata.prh.fi/tr/v1/publicnotices?totalResults=false&maxResults=1000' + \
        '&noticeRegistrationFrom=2011-01-01&noticeRegistrationTo=2018-12-31' + \
        '&resultsFrom={}'.format(i)
    r = requests.get(url)
    if r.status_code == 429:
        print('status 429')
        print('dumping to csv')
        records.to_csv('recs{}.csv'.format(i))
        break
    else:
        try:
            recs = r.json()['results']
            records = records.append(pd.DataFrame.from_records(recs).iloc[:,[0,1,2,3,5]])
            i += 1000
            if i % 100000 == 0:
                print('dumping to csv')
                records.to_csv('recs{}.csv'.format(i))
                records = pd.DataFrame()
        except JSONDecodeError:
            print('jsondecodeerror')
            continue

Trying: 1500000
status 429
dumping to csv


In [ ]:
import glob
import pandas as pd
import numpy as np
# get data file names

path='C:/Users/lauri/koodi/vero'

filenames = glob.glob(path + "/recs*.csv")
print(filenames)
dfs = []

for filename in filenames:
    
    dfs.append(pd.read_csv(filename, index_col=0))

records = pd.concat(dfs, ignore_index=True)

records.entryCodes = [eval(val) for val in records.entryCodes]

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
records = records.join(pd.DataFrame(mlb.fit_transform(records.entryCodes), 
                                    columns=mlb.classes_, index=records.index))
records.set_index('recordNumber', inplace=True)

mask = records.iloc[:,4:].columns[records.iloc[:,4:].sum() < 100]
records['MUUMUU'] = records[mask].sum(1)
records = records.drop(mask, 1)
records.to_csv('records_processed.csv')

In [3]:
df = pd.DataFrame()
for i in range(6015, 8000):
    url = 'http://www.protestilista.com/?page={}'.format(i)
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    ids = [a['href'][7:] for a in soup.select('a[class=""]')]
    info = [a.text.strip() for a in soup.select('span[class="color-white-mute"]')]
    debtors = info[::3]
    sums = info[1::3]
    dates = info[2::3]
    df = df.append(pd.DataFrame([ids, debtors, sums, dates]).T)
    print('Got ', i)
    if i % 1000 == 0:
        df.to_csv('protestit2_{}.csv'.format(i))

Got  6015
Got  6230


In [4]:
df.to_csv('protestit2_2.csv')